In [ ]:
# coding: utf-8
import gc
import multiprocessing
import sys
import time
import numpy as np
import pandas as pd
import textblob
from functools import partial
from tqdm import tqdm, tqdm_pandas

In [ ]:
WORKER_NUMBER = 14 # change here [1-15]
MAX_WORKER_NUMBER = 15

In [ ]:
ls ../input

In [ ]:
# params
n_workers = 11
from_lang = 'en'
to = 'ja'
text_column = 'comment_text'

# paths
INPUT_TRAIN_PATH = '../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv'
OUTPUT_TRAIN_PATH = 'train_ja.pkl'

In [ ]:
def translate(x, from_lang='en', to='ja'):
    try:
        return ''.join(textblob.TextBlob(x).translate(from_lang=from_lang, to=to))
    except:
        return x
    
def map_translate(df):
    tqdm.pandas(tqdm())
    translated = df.progress_map(lambda x: translate(x, from_lang=from_lang, to=to))
    return translated

In [ ]:
# load
train = pd.read_csv(INPUT_TRAIN_PATH)
# test = pd.read_csv(INPUT_TEST_PATH)
len_train = len(train)
# train = pd.concat([train, test], sort=True)
print(train.shape)
train.head(3)

In [ ]:
# split by worker number
n_samples = (len_train // MAX_WORKER_NUMBER) + 1
print(n_samples)
train = train.iloc[(WORKER_NUMBER - 1) * n_samples: min(WORKER_NUMBER * n_samples, len_train)]
train.head(3)

In [ ]:
# translate
with multiprocessing.Pool(processes=n_workers) as p:
    split_dfs = np.array_split(train[text_column], n_workers)
    pool_results = p.map(map_translate, split_dfs)
train[text_column] = pd.concat(pool_results, sort=True)

In [ ]:
train.head(3)

In [ ]:
# save
# test = train.iloc[len_train:].reset_index(drop=True)
# train = train.iloc[:len_train].reset_index(drop=True)
train.to_pickle(OUTPUT_TRAIN_PATH)
# test.to_pickle(OUTPUT_TEST_PATH)